# NEAT examples — XOR, parameter sweep, and CartPole

This notebook contains runnable examples for the XOR problem, a small parameter sweep, and a short CartPole run using neat-python and gymnasium.
## How to run this notebook

- Local (recommended for development): activate the project's virtual environment and run cells in order using Jupyter or execute the notebook non-interactively:

```bash
python -m venv .venv
./.venv/Scripts/Activate.ps1               # PowerShell (Windows)
python -m pip install -r requirements.txt  
```

- Google Colab: run the first code cell ("Google Colab setup") and follow its prompts. If packages are installed you may need to restart the runtime, then run cells top-to-bottom.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RichardJPovinelli/Evolutionary_Computation_Course/blob/main/NEAT_demo.ipynb)


- Outputs: experiment outputs are written into `results/` and `sweep_results/` (JSON summaries and checkpointer files). Keep these folders under version control only for reproducible examples.

## Setup environment

In [1]:
# Google Colab setup: install required packages when running in Colab
IN_COLAB = False
try:
    import google.colab
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    print('Running in Google Colab — installing dependencies...')
    %pip install neat-python
    %pip install gymnasium[classic-control]
    # If you need additional system packages, install them here (apt-get)
    print('Dependencies installed; you may need to restart the runtime')
else:
    print('Not running in Colab — skipping Colab-specific setup')


Not running in Colab — skipping Colab-specific setup


In [2]:
# Imports
import os
import json
import random
import statistics
import itertools

import gymnasium as gym
import neat
from neat.nn import FeedForwardNetwork
from neat.reporting import BaseReporter

print('Imports OK')

Imports OK


In [3]:
#@title Create default NEAT config files in the notebook if they are missing
import os

xor_cfg = '''[NEAT]
fitness_criterion=max
fitness_threshold=3.9
pop_size=100
reset_on_extinction=False

[DefaultGenome]
activation_default=sigmoid
activation_mutate_rate=0.0
activation_options=sigmoid
aggregation_default=sum
aggregation_mutate_rate=0.0
aggregation_options=sum
bias_init_mean=0.0
bias_init_stdev=1.0
bias_max_value=5.0
bias_min_value=-5.0
bias_mutate_power=0.5
bias_mutate_rate=0.7
bias_replace_rate=0.1
compatibility_disjoint_coefficient=1.0
compatibility_weight_coefficient=0.5
conn_add_prob=0.5
conn_delete_prob=0.3
enabled_default=True
enabled_mutate_rate=0.01
feed_forward=True
initial_connection=partial_direct 0.5
node_add_prob=0.2
node_delete_prob=0.05
num_hidden=0
num_inputs=2
num_outputs=1
response_init_mean=1.0
response_init_stdev=0.0
response_max_value=1.0
response_min_value=1.0
response_mutate_rate=0.0
response_mutate_power=0.0
response_replace_rate=0.0
weight_init_mean=0.0
weight_init_stdev=1.0
weight_max_value=5.0
weight_min_value=-5.0
weight_mutate_power=0.5
weight_mutate_rate=0.8
weight_replace_rate=0.1

[DefaultSpeciesSet]
compatibility_threshold=3.0

[DefaultStagnation]
species_fitness_func=max
max_stagnation=15
species_elitism=1

[DefaultReproduction]
elitism=2
survival_threshold=0.2
'''

sweep_template = '''[NEAT]
fitness_criterion     = max
fitness_threshold     = 3.9
pop_size              = {pop_size}
reset_on_extinction   = False

[DefaultGenome]
activation_default      = sigmoid
activation_mutate_rate  = 0.0
activation_options      = sigmoid
aggregation_default     = sum
aggregation_mutate_rate = 0.0
aggregation_options     = sum
bias_init_mean          = 0.0
bias_init_stdev         = 1.0
bias_max_value          = 5.0
bias_min_value          = -5.0
bias_mutate_power       = 0.5
bias_mutate_rate        = 0.7
bias_replace_rate       = 0.1
compatibility_disjoint_coefficient = 1.0
compatibility_weight_coefficient   = 0.5
conn_add_prob           = 0.5
conn_delete_prob        = 0.3
enabled_default         = True
enabled_mutate_rate     = 0.01
feed_forward            = True
initial_connection      = partial_direct 0.5
node_add_prob           = 0.2
node_delete_prob        = 0.05
num_hidden              = 0
num_inputs              = 2
num_outputs             = 1
response_init_mean      = 1.0
response_init_stdev     = 0.0
response_max_value      = 1.0
response_min_value      = 1.0
response_mutate_rate    = 0.0
response_mutate_power   = 0.0
response_replace_rate    = 0.0
weight_init_mean        = 0.0
weight_init_stdev       = 1.0
weight_max_value        = 5.0
weight_min_value        = -5.0
weight_mutate_power     = 0.5
weight_mutate_rate      = 0.8
weight_replace_rate     = 0.1

[DefaultSpeciesSet]
compatibility_threshold = {compat_threshold}

[DefaultStagnation]
species_fitness_func = max
max_stagnation       = 12
species_elitism      = 1

[DefaultReproduction]
elitism            = 2
survival_threshold = 0.2
'''

cartpole_cfg = '''[NEAT]
fitness_criterion     = max
fitness_threshold     = 475.0
pop_size              = 150
reset_on_extinction   = False

[DefaultGenome]
activation_default      = tanh
activation_mutate_rate  = 0.0
activation_options      = tanh
aggregation_default     = sum
aggregation_mutate_rate = 0.0
aggregation_options     = sum
bias_init_mean          = 0.0
bias_init_stdev         = 1.0
bias_max_value          = 5.0
bias_min_value          = -5.0
bias_mutate_power       = 0.5
bias_mutate_rate        = 0.7
bias_replace_rate       = 0.1
compatibility_disjoint_coefficient = 1.0
compatibility_weight_coefficient   = 0.5
conn_add_prob           = 0.5
conn_delete_prob        = 0.3
enabled_default         = True
enabled_mutate_rate     = 0.01
feed_forward            = True
initial_connection      = full_direct
node_add_prob           = 0.2
node_delete_prob        = 0.05
num_hidden              = 0
num_inputs              = 4
num_outputs             = 1
response_init_mean      = 1.0
response_init_stdev     = 0.0
response_max_value      = 1.0
response_min_value      = 1.0
response_mutate_rate    = 0.0
response_mutate_power   = 0.0
response_replace_rate   = 0.0
weight_init_mean        = 0.0
weight_init_stdev       = 1.0
weight_max_value        = 5.0
weight_min_value        = -5.0
weight_mutate_power     = 0.5
weight_mutate_rate      = 0.8
weight_replace_rate     = 0.1

[DefaultSpeciesSet]
compatibility_threshold = 3.0

[DefaultStagnation]
species_fitness_func = max
max_stagnation       = 15
species_elitism      = 1

[DefaultReproduction]
elitism            = 2
survival_threshold = 0.2
'''

def write_if_missing(path, text):
    if not os.path.exists(path):
        with open(path, 'w', encoding='utf-8') as fh:
            fh.write(text)
        print(f'Wrote {path}')
    else:
        print(f'{path} already exists, skipping')

write_if_missing('xor_config.cfg', xor_cfg)
write_if_missing('sweep_config_template.cfg', sweep_template)
write_if_missing('cartpole_config.cfg', cartpole_cfg)
print('Config files checked/created (if missing).')

xor_config.cfg already exists, skipping
sweep_config_template.cfg already exists, skipping
cartpole_config.cfg already exists, skipping
Config files checked/created (if missing).


## XOR problem: inputs, labels, and evaluation

This section contains a minimal XOR experiment to demonstrate how NEAT evolves small feedforward networks.

- Dataset: four input pairs with binary targets (0/1). The network is evaluated using mean-squared-like fitness (1.0 - squared error per case) summed over examples.
- What to expect: with small populations and few generations the run may not find a perfect solution; check the statistics reporter output and saved checkpoints in `sweep_results/`.

Tips:
- Reduce 'N_XOR_GENERATIONS` in tests to run faster.

### XOR Constants

In [4]:
XOR_ACTIVATION_DEFAULT = 'sigmoid'
XOR_ACTIVATION_OPTIONS = ['sigmoid'] # ['tanh', 'sigmoid', 'relu]
XOR_POP_SIZE = 100
N_XOR_GENERATIONS = 100

xor_input_pairs = [(0.0, 0.0), (0.0, 1.0), (1.0, 0.0), (1.0, 1.0)]
xor_expected_outputs = [0.0, 1.0, 1.0, 0.0]

## Main XOR Code

In [ ]:
def compute_genome_xor_fitness(genome, config):
    network = FeedForwardNetwork.create(genome, config)
    fitness = 0.0
    for inputs, target in zip(xor_input_pairs, xor_expected_outputs):
        output = network.activate(inputs)[0]
        fitness += 1.0 - (output - target) ** 2
    return fitness

def evaluate_population_xor(genomes, config):
    for _, genome in genomes:
        genome.fitness = compute_genome_xor_fitness(genome, config)

# Run action single XOR experiment (short smoke test)
config_path = 'xor_config.cfg'
if not os.path.exists(config_path):
    raise FileNotFoundError(f"{config_path} not found. Add it to the code folder.")

config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)
config.pop_size = XOR_POP_SIZE
config.genome_config.activation_default = XOR_ACTIVATION_DEFAULT
config.genome_config.activation_options = XOR_ACTIVATION_OPTIONS

population = neat.Population(config)
population.add_reporter(neat.StdOutReporter(True))
statistics_reporter = neat.StatisticsReporter()
population.add_reporter(statistics_reporter)
os.makedirs('sweep_results', exist_ok=True)
population.add_reporter(neat.Checkpointer(5, filename_prefix='sweep_results/xor-'))
best_genome = population.run(evaluate_population_xor, n=N_XOR_GENERATIONS)
if best_genome is None:
    print('\nNo best_genome produced by population run')
else:
    print('\nBest genome:', best_genome)
    network = FeedForwardNetwork.create(best_genome, config)
    for inputs, target in zip(xor_input_pairs, xor_expected_outputs):
        network_output = network.activate(inputs)[0]
        print(f"Input={inputs} -> output={network_output:.3f}, target={target}")



## Parameter sweep 
A small grid search that writes temporary cfgs into `results/` and records summary statistics.

This cell runs a small grid search over `pop_size` and `compat_threshold`. For each grid point we run two replicate experiments (two random seeds) and record:

- `best_fitness_mean` and `best_fitness_stdev` — mean and population standard deviation of the best fitness from the replicates
- `avg_species_last_gen` — average number of species observed in the final generation across replicates

Notes:
- The sweep writes generated config files to `sweep_results/` and a summary JSON at the end.
- For longer sweeps, increase `SWEEP_N_GENERATIONS` and add more replicate seeds; expect runtime to grow linearly with runs * reps.
- Use the checkpointer files to resume interrupted runs (they are saved under `sweep_results/`).

In [6]:
SWEEP_ACTIVATION_OPTIONS = ['sigmoid', 'tanh', 'relu']
SWEEP_POP_SIZES = [50, 100, 200]
SWEEP_SPECIES_THRESHOLDS = [1.0, 2.5, 3.0]
SWEEP_N_GENERATIONS = 100
SWEEP_ACTIVATION_DEFAULT = 'sigmoid' 

In [7]:
# Helper to run one experiment (returns best fitness and species counts)
def run_single_experiment(config_path, max_generations=20):
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)
    config.genome_config.activation_default = XOR_ACTIVATION_DEFAULT
    config.genome_config.activation_options = XOR_ACTIVATION_OPTIONS
    population = neat.Population(config)
    species_counts = []

    class SpeciesSpy(BaseReporter):
        def post_evaluate(self, config, population, species, best_genome):
            species_counts.append(len(species.species))

    population.add_reporter(SpeciesSpy())
    best_genome = population.run(evaluate_population_xor, n=SWEEP_N_GENERATIONS)
    if best_genome is None:
        return 0.0, species_counts
    return best_genome.fitness, species_counts


# Small sweep (fast settings for smoke test)
os.makedirs('sweep_results', exist_ok=True)
# compat_threshold is the distance threshold for speciation
param_grid = {'pop_size': SWEEP_POP_SIZES, 'compat_threshold': SWEEP_SPECIES_THRESHOLDS}
sweep_results = []
sweep_run_id = 0
for pop_size, compat_threshold in itertools.product(param_grid['pop_size'], param_grid['compat_threshold']):
    sweep_run_id += 1
    generated_cfg_text = open('sweep_config_template.cfg', encoding='utf-8').read().format(pop_size=pop_size, compat_threshold=compat_threshold)
    config_path = f'sweep_results/run_{sweep_run_id}_cfg.cfg'
    with open(config_path, 'w', encoding='utf-8') as f:
        f.write(generated_cfg_text)
    replicate_runs = []
    for random_seed in [0, 1]:
        random.seed(random_seed)
        fitness, species_counts = run_single_experiment(config_path, max_generations=20)
        replicate_runs.append({'fitness': fitness, 'species_counts': species_counts})
    best_fitnesses = [r['fitness'] for r in replicate_runs]
    avg_species_last_gen = statistics.mean([r['species_counts'][-1] for r in replicate_runs]) if all(r['species_counts'] for r in replicate_runs) else 0
    sweep_results.append({
        'pop_size': pop_size,
        'compat_threshold': compat_threshold,
        'best_fitness_mean': statistics.mean(best_fitnesses),
        'best_fitness_stdev': statistics.pstdev(best_fitnesses),
        'avg_species_last_gen': avg_species_last_gen,
    })
    print(f"[population={pop_size}, compat={compat_threshold}] best_mean={statistics.mean(best_fitnesses):.3f} ± {statistics.pstdev(best_fitnesses):.3f}; species@last={avg_species_last_gen:.1f}")
with open('sweep_results/sweep_summary.json', 'w', encoding='utf-8') as f:
    json.dump(sweep_results, f, indent=2)
print('\nSaved sweep_results to sweep_results/sweep_summary.json')

[population=50, compat=1.0] best_mean=3.144 ± 0.066; species@last=21.5
[population=50, compat=2.5] best_mean=3.834 ± 0.090; species@last=4.0
[population=50, compat=3.0] best_mean=3.687 ± 0.225; species@last=1.5
[population=100, compat=1.0] best_mean=3.613 ± 0.372; species@last=49.0
[population=100, compat=2.5] best_mean=3.949 ± 0.044; species@last=6.0
[population=100, compat=3.0] best_mean=3.969 ± 0.007; species@last=2.5
[population=200, compat=1.0] best_mean=3.308 ± 0.025; species@last=102.5
[population=200, compat=2.5] best_mean=3.944 ± 0.038; species@last=7.5
[population=200, compat=3.0] best_mean=3.959 ± 0.000; species@last=3.5

Saved sweep_results to sweep_results/sweep_summary.json


## CartPole evaluation and run

The CartPole section evaluates genomes on the `CartPole-v1` environment using gymnasium. Important points:

- The evaluation returns the average episode return across `episodes` runs. Higher is better.
- We use a simple discrete action policy: the network outputs a single scalar which we threshold at 0.0 to produce action 0 or 1.
- CartPole is stochastic. Use several episodes to get robust estimates.

Tuning tips:
- Reduce `episodes` and `max_steps` for quick smoke tests.
- If you see unstable training, try increasing population size or the `compat_threshold` parameter in the NEAT config.

### CartPole Constants

In [8]:
CART_POP_SIZE = 50
CART_ACTIVATION_DEFAULT = 'sigmoid'
CART_ACTIVATION_OPTIONS = ['sigmoid'] # ['tanh', 'sigmoid', 'relu']

## CartPole Main Code

In [9]:
def choose_cartpole_action(network, observation):
    network_output = network.activate(observation)[0]
    return 1 if network_output > 0.0 else 0

def compute_genome_cartpole_average_return(genome, config, episodes=2, max_steps=500):
    env = gym.make('CartPole-v1')
    network = FeedForwardNetwork.create(genome, config)
    total_return = 0.0
    for _ in range(episodes):
        observation, _ = env.reset()
        episode_return = 0.0
        for _ in range(max_steps):
            action = choose_cartpole_action(network, observation)
            observation, r, terminated, truncated, _ = env.step(action)
            episode_return += float(r)
            if terminated or truncated:
                break
        total_return += episode_return
    env.close()
    return total_return / episodes

def evaluate_population_cartpole(genomes, config):
    for _, genome in genomes:
        genome.fitness = compute_genome_cartpole_average_return(genome, config)


# Run CartPole (short test)
config_path = 'cartpole_config.cfg'
if not os.path.exists(config_path):
    raise FileNotFoundError(f"{config_path} not found. Add it to the code folder.")

config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)
config.pop_size = CART_POP_SIZE
config.genome_config.activation_default = CART_ACTIVATION_DEFAULT
config.genome_config.activation_options = CART_ACTIVATION_OPTIONS
population = neat.Population(config)

population.add_reporter(neat.StdOutReporter(True))
statistics_reporter = neat.StatisticsReporter()
population.add_reporter(statistics_reporter)
os.makedirs('sweep_results', exist_ok=True)
population.add_reporter(neat.Checkpointer(5, filename_prefix='sweep_results/cartpole-'))
best_genome = population.run(evaluate_population_cartpole, n=20)
if best_genome is None:
    print('\nNo best_genome produced by CartPole population run')
else:
    print('\nWinner avg fitness:', compute_genome_cartpole_average_return(best_genome, config, episodes=3))


 ****** Running generation 0 ****** 

Population's average fitness: 9.36000 stdev: 0.64062
Best fitness: 10.50000 - size: (1, 4) - species 1 - id 18
Average adjusted fitness: 0.544
Mean genetic distance 1.203, standard deviation 0.426
Population of 50 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    50     10.5    0.544     0
Total extinctions: 0
Generation time: 0.018 sec

 ****** Running generation 1 ****** 

Population's average fitness: 9.36000 stdev: 0.48000
Best fitness: 10.50000 - size: (2, 6) - species 1 - id 84
Average adjusted fitness: 0.430
Mean genetic distance 1.455, standard deviation 0.411
Population of 50 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    50     10.5    0.430     1
Total extinctions: 0
Generation time: 0.017 sec (0.018 average)

 ****** Running generation 2 ****** 

Population's average fitness: 9.24000 stdev: 0.